In [8]:
import pandas as pd
from joblib import dump
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedKFold 

## Mejora del modelo seleccionado regresión logística

En esta parte no se hará mucho énfasis en la preparación de los datos ya que en el notebook model_v1.ipynb se hace de manera mas detallada aquí la idea es mejorar le modelo escogido

In [3]:
SEED = 137 # semilla aleatoria para la reproducibilidad del análisis

In [4]:
df = pd.read_csv("data/data_clean.csv", index_col=0)

In [5]:
df = df.drop(columns=["numero_aviso", # esta variable solo indica el identificador incidente
                      "marca", # esta variable tiene baja correlación y alta cardinalidad, es un identificador
                      "accion_modelo", # variable que vien de un módelo anterior
                      "linea", # es un identificador
                      "nombre" # esta variable tien mucha correlación con el codigo iris
                     ])

In [6]:
df["grupo"] = df["grupo"].astype(str)
df["subgrupo"] = df["subgrupo"].astype(str)

In [7]:
y_temp = pd.get_dummies(df["accion"])

In [9]:
y = y_temp["cambiar"].values
y

array([1, 1, 1, ..., 1, 1, 0], dtype=uint8)

In [10]:
df = df.drop(columns=["accion"]) # ahora esta variable es denominada y

In [11]:
df_model = pd.get_dummies(df)

In [12]:
X = df_model.values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=SEED)

### Optimización de hiperparámetros con GridSearchCV

In [16]:
lr_class = LogisticRegression(max_iter=500)
grid = {'C':[0.5, 1, 1.5, 2, 2.5],
        "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky']}

In [ ]:
lr_class_grid = GridSearchCV(lr_class, # modelo
                             grid, # grid
                             n_jobs=8, # numéro de hilos
                             cv=10 # validación cruzada iterativa)
lr_class_grid_fit =  lr_class_grid.fit(X_train, y_train)

pd.concat([pd.DataFrame(lr_class_grid_fit.cv_results_["params"]),
           pd.DataFrame(lr_class_grid_fit.cv_results_["mean_test_score"], 
                        columns=["Accuracy"])], axis=1)
                             
pd.DataFrame(rf_class_grid_fit.cv_results_["mean_test_score"], 
                        columns=["Accuracy"], axis=1)

In [ ]:
# mejor modelo de los anteriores
print(f'Best parameters: {lr_class_grid_fit.best_params_}')
print(f'Best score: {lr_class_grid_fit.best_score_}')

In [ ]:
y_pred_lr_grid = lr_class_grid_fit.predict(X_test)

print('##-- Random Forest Grid Search & CV --## ')
print(f'Acuraccy: {accuracy_score(y_test, y_pred_lr_grid)}')
print(f'Precision: {precision_score(y_test, y_pred_lr_grid, average="macro")}')
print(f'Recall: {recall_score(y_test, y_pred_lr_grid, average="macro")}')
print(confusion_matrix(y_test, y_pred_lr_grid))

In [ ]:
# guardar le modelo para poner en el demo
dump(lr_class, 'model/lr_class.joblib') 